# Actor-critic methods

## Principe d'actor critic

## A2C (l'algorithme que l'on va implémenter)


## Appliquons cela sur un petit jeu simple, Mountain Car 

Doc et code source <a href="https://github.com/openai/gym/wiki/MountainCar-v0">ici</a>

Ce jeu est disponible dans l'environnement gym. Le principe est plutôt simple: faire grimper une colline à une voiture. La voiture n'est pas capable de monter en une seule fois: il faut avancer et reculer au fur et à mesure pour prendre de l'élan et réussir à monter. Le jeu se termine quand la voiture a atteint le drapeau, ou si la voiture n'a pas réussi après 200 itérations

Ce jeu est modélisé par des états codés comme un vecteur en 2 dimensions modélisant 2 mesures physiques: position de la voiture en x, et vitesse. Trois actions possible: bouger d'un cran vers la droite, bouger d'un cran vers la gauche, ne rien faire.

<img src="images/mountaincar.gif" width="500">

Configurons l'environnement Mountain Car-V0 avec gym: